# Evaluating BridgeStan Speed

This notebook is meant to evaluate the speed of BridgeStan relative to [issue #190](https://github.com/roualdes/bridgestan/issues/190). The goal is to offer users direct access Stan's methods via both numpy arrays and ctypes.  The question is which method is fastest and which method will be reasonable to maintain into the future.  For speed comparisons, BridgeStan's `log_density_gradient(...)` will serve as the baseline.

There are two strategies (to offering APIs for numpy arrays and ctypes) being evaluated.  The function `log_density_gradient_proposed(...)` uses `array_ptr` to differentiate between numpy arrays and ctypes `double*`s with an implicit if statement. An alternative solution is to expose two functions, one specifically for ctypes and one for numpy arrays.  The function `log_density_gradient_ctypes(...)` offers direct access to ctypes, while `log_density_gradient_alternative(...)` deals with numpy arrays, but internally just calls `log_density_gradient_ctypes(...)`.

### Clone and Install BridgeStan

In [7]:
# start fresh?
!rm -rf ./bridgestan

In [8]:
!git clone --recurse-submodules --shallow-submodules --depth=1 https://github.com/roualdes/bridgestan.git
!cd ./bridgestan && git fetch origin python/enable-ctypes-double-pointers:python/enable-ctypes-double-pointers && git checkout python/enable-ctypes-double-pointers

Cloning into 'bridgestan'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 184 (delta 3), reused 109 (delta 1), pack-reused 0
Receiving objects: 100% (184/184), 145.52 KiB | 1.09 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Submodule 'stan' (https://github.com/stan-dev/stan) registered for path 'stan'
Cloning into '/Users/ez/bridgestan-speed/bridgestan/stan'...
remote: Enumerating objects: 620, done.        
remote: Counting objects: 100% (620/620), done.        
remote: Compressing objects: 100% (519/519), done.        
remote: Total 620 (delta 148), reused 299 (delta 77), pack-reused 0        
Receiving objects: 100% (620/620), 4.63 MiB | 3.36 MiB/s, done.
Resolving deltas: 100% (148/148), done.
remote: Total 0 (delta 0), reused 0 (delta 0), pack-reused 0
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100%

In [9]:
!cd ./bridgestan && pip install ./python

Processing ./python
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for bridgestan: filename=bridgestan-2.2.2-py3-none-any.whl size=11744 sha256=f7e1d856ec052bba58320a306b6c202cb0916dd37790ebfc80fd14a40507aa08
  Stored in directory: /private/var/folders/10/vhgkp_1x0p310y0lw2d5mx8h0000gn/T/pip-ephem-wheel-cache-5_21ew09/wheels/c0/af/43/4c10ee4e3df14332c19c8b6a53ff3b068ac02d4af336917a59
Successfully built bridgestan
  Attempting uninstall: bridgestan
    Found existing installation: bridgestan 2.2.2
    Uninstalling bridgestan-2.2.2:
      Successfully uninstalled bridgestan-2.2.2

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


### Clone and Install ExperimentalHMC (for testing direct ctypes access)

In [2]:
!git clone git@github.com:roualdes/experimentalHMC.git
!cd ./experimentalHMC && git fetch origin start && git checkout start
!cd ./experimentalHMC && pip install .

fatal: destination path 'experimentalHMC' already exists and is not an empty directory.
From github.com:roualdes/experimentalHMC
 * branch            start      -> FETCH_HEAD
Already on 'start'
Your branch is up to date with 'origin/start'.
Processing /Users/ez/bridgestan-speed/experimentalHMC
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experimentalhmc: filename=experimentalhmc-0.0.1-cp311-cp311-macosx_12_0_x86_64.whl size=28055 sha256=d9cd279c0e58ce33d3a4e0831af81f5f6d532ff24519b14abd8f18f10e60fac5
  Stored in directory: /private/var/folders/10/vhgkp_1x0p310y0lw2d5mx8h0000gn/T/pip-ephem-wheel-cache-uvqynm6t/wheels/db/d5/8f/6d7c68188c07281292ae0013dfd8cb0853d40aad60344fb904
Successfully built experimentalhmc
  Attempting uninstall: experimentalhmc
    Found existing installation: experimentalhmc 0.0.1
    Uninstalling experimentalhmc-0.0.1:
      Successfully uninstalled exper

### Evaluate Numpy Access

In [10]:
from pathlib import Path

import bridgestan as bs
import numpy as np

bs.set_bridgestan_path(Path.home() / "bridgestan")

In [11]:
model = "logistic"

gm = bs.StanModel(f"test_models/{model}/{model}.stan",
                  data = f"test_models/{model}/{model}.data.json")

dims = gm.param_unc_num()
R = 1_000
out = np.zeros(dims)

/Users/ez/venvs/py3/lib/python3.11/site-packages/bridgestan/model.py:127: UserWarning: Loading a shared object /Users/ez/bridgestan-speed/test_models/logistic/logistic_model.so that has already been loaded.
If the file has changed since the last time it was loaded, this load may not update the library!
  warnings.warn(


In [12]:
%%timeit -n 10
for r in range(R):
    q = np.random.normal(size = dims)
    gm.log_density_gradient(q, out = out)

88.3 ms ± 3.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit -n 10
for r in range(R):
    q = np.random.normal(size = dims)
    gm.log_density_gradient_proposed(q, out = out)

82.3 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit -n 10
for r in range(R):
    q = np.random.normal(size = dims)
    gm.log_density_gradient_alternative(q, out = out)

84.9 ms ± 700 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Evaluate Ctypes Access

In [15]:
import experimentalhmc as ehmc
from numpy.ctypeslib import as_array

In [16]:
def bridgestan_log_density_gradient_wrapper(bsm):
    dim = bsm.param_unc_num()
    def bsm_c_wrapper(position, gradient):
        ld, _ = bsm.log_density_gradient(as_array(position, shape = (dims,)), 
                                      out = as_array(gradient, shape = (dims,)))
        return ld
    return bsm_c_wrapper

def bridgestan_log_density_gradient_proposed_wrapper(bsm):
    dim = bsm.param_unc_num()
    def bsm_c_wrapper(position, gradient):
        ld, _ = bsm.log_density_gradient_proposed(position, out = gradient)
        return ld
    return bsm_c_wrapper
    
def bridgestan_log_density_gradient_ctypes_wrapper(bsm):
    dim = bsm.param_unc_num()
    def bsm_c_wrapper(position, gradient):
        ld = bsm.log_density_gradient_ctypes(position, gradient)
        return ld
    return bsm_c_wrapper

In [17]:
ldg = bridgestan_log_density_gradient_wrapper(gm)
ldg_proposed = bridgestan_log_density_gradient_proposed_wrapper(gm)
ldg_ctypes = bridgestan_log_density_gradient_ctypes_wrapper(gm)

In [18]:
stan = ehmc.Stan(dims, ldg)
stan_proposed = ehmc.Stan(dims, ldg_proposed)
stan_ctypes = ehmc.Stan(dims, ldg_ctypes)

In [19]:
%%timeit -n 10
for m in range(1000):
    stan.sample()

2.38 s ± 31.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%timeit -n 10
for m in range(1000):
    stan_proposed.sample()

1.88 s ± 84.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%%timeit -n 10
for m in range(1000):
    stan_ctypes.sample()

2.01 s ± 102 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
